In [1]:
import SimpleITK as sitk
import six
import matplotlib.pyplot as plt
import numpy as np
np.seterr(all="ignore")
import os
import nibabel as nib
from radiomics import featureextractor 
from IPython.core.display import display, HTML
from IPython.utils import io
import logging
from util import pyvis, make_feature_map
from scipy.stats import wilcoxon
import pandas as pd
import nrrd
HTML('''
<style>
.jupyter-matplotlib {
    background-color: #000;
}

.widget-label, .jupyter-matplotlib-header{
    color: #fff;
}

.jupyter-button {
    background-color: #333;
    color: #fff;
}
</style>
''')

In [2]:
res = ['07','20','31']
nres = ['05','22','28']
ids = ['07','20','31','05','22','28']

In [3]:

%matplotlib widget
visualize = 0
level = 0
window = 500
m1 = level-window/2
m2 = level+window/2
slices = []
segs = []
qsms = []
skipcount = 0
for j in np.arange(len(ids)):
    V = nib.load('/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_'+ids[j]+'.nii.gz')
    M = nib.load('/home/ali/RadDBS-QSM/data/nii/chh/seg/'+ids[j]+'.nii.gz')
    qsms.append(V.get_fdata())
    segs.append(M.get_fdata())
    rows,cols,slices = V.shape
    img_shape = (slices,rows,cols)
    plots = np.zeros(img_shape)


    for i in np.arange(slices):
        pix = V.get_fdata()[:,:,i]
        m = V.get_fdata() != -250
        pix[pix<m1]=m1
        pix[pix>m2]=m2
        # plots[i,:,:] = m[:,:,i]*pix

    if visualize == 1:
        pyvis(np.hstack((np.flip(V.get_fdata().T),np.flip(100*M.get_fdata().T))),10,10)




In [4]:
df = pd.read_csv('/home/ali/RadDBS-QSM/data/xlxs/updrs_iii_chh.csv')
n_rois = 6
n_cases = 37
n_features = 1596
X0_gt = np.load('/home/ali/RadDBS-QSM/data/npy/rp/X0_gt_chh_rois_rp.npy')
R = np.load('/home/ali/RadDBS-QSM/data/npy/rp/R_chh_rp.npy')
K = np.load('/home/ali/RadDBS-QSM/data/npy/rp/K_chh_rp.npy')
K_all_c = K.reshape(n_cases,n_rois,n_features-1)[:,0:4,:]
K_all_c = K_all_c[0,0,:].reshape(-1,1)
X_all_c = X0_gt.reshape(n_cases,n_rois,n_features)[:,0:4,:]
# Patient IDs
subject_id = np.asarray(df[df.columns[0]])[1:]
# Only extract ROI if it is present in all cases
seg_labels_all = segs[0]
s_directory = '/home/ali/RadDBS-QSM/data/nii/chh/orig/seg/'
s_directory = os.listdir(s_directory)
s_directory = sorted(s_directory)
case_number = np.zeros_like(np.asarray(s_directory))
for i in range(n_cases):
    case_number[i] = float(s_directory[i][:2])
subject_id_corr = subject_id[np.in1d(subject_id,case_number)]
pre_updrs_iii_off =  np.asarray(df[df.columns[3]][np.hstack((False,np.in1d(subject_id,subject_id_corr)))])                                
pre_updrs_iii_on =  np.asarray(df[df.columns[4]][np.hstack((False,np.in1d(subject_id,subject_id_corr)))])
post_updrs_iii_off =  np.asarray(df[df.columns[6]][np.hstack((False,np.in1d(subject_id,subject_id_corr)))])
per_change = (np.asarray(pre_updrs_iii_off).astype(float)-np.asarray(post_updrs_iii_off).astype(float))/(np.asarray(pre_updrs_iii_off).astype(float))

In [5]:
logger = logging.getLogger("radiomics")
logger.setLevel(logging.CRITICAL)
reextract = 0
rewrite = 0
# Extract predictive features only
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.disableAllImageTypes()
extractor.disableAllFeatures()
extractor.enableImageTypeByName('Original')
extractor.enableImageTypeByName('Gradient')
extractor.enableImageTypeByName('Square')
extractor.enableImageTypeByName('Wavelet')
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('glcm')
extractor.enableFeatureClassByName('gldm')
extractor.enableFeatureClassByName('glszm')
print('Enabled features:\n\t', extractor.enabledFeatures)
x_row = []
Phi = []
keylib = []
roilib = []
roi_names = np.asarray(['Right substantia nigra','Right subthalamic nucleus',
                            'Left subthalamic nucleus', 'Left substantia nigra', 'Right dentate nucleus', 'Left dentate nucleus'])
phi_predict = ['wavelet-LLL_gldm_GrayLevelNonUniformity','gradient_glcm_SumEntropy','square_firstorder_Variance','wavelet-HHL_glszm_SizeZoneNonUniformity']
phic = '\t'.join(phi_predict)
# Convert to SITK images and extract features
if reextract == 1:
    for j in np.arange(len(ids)):
        qsm_sitk = sitk.GetImageFromArray(qsms[j]) 
        seg = np.asarray(segs[j].astype(int))
        seg_sitk = sitk.GetImageFromArray(seg) 
        for k in np.arange(1,len(roi_names)+1):
            with io.capture_output() as captured:
                featureVector = extractor.execute(qsm_sitk,seg_sitk,voxelBased=True,label=k);
                fv_count = 0
                Phi.append(featureVector)
            print('Extracted feature maps for subject',ids[j],'ROI',str(k))
            for key, value in six.iteritems(featureVector):
                if isinstance(value, sitk.Image):  # Feature map
                    if key in phic and rewrite == 1:
                        print('Writing '+key+' map')
                        sitk.WriteImage(value,key +'_'+ids[j]+'_'+str(k)+'.nrrd', True)
                        sitk.WriteImage(seg_sitk,key +'seg_'+ids[j]+'_'+str(k)+'.nrrd', True)
                if key == 'diagnostics_Mask-original_BoundingBox':
                    lims = value
                    # Assuming kernel radius of 3, slide of 1, and padding of 0
                    sitk.WriteImage(seg_sitk[lims[0]-1:lims[0]+lims[3]+1,lims[1]-1:lims[1]+lims[4]+1,lims[2]-1:lims[2]+lims[5]+1], 'seg_crop_' + ids[j] +'_'+str(k)+ '.nrrd',True)
                    sitk.WriteImage(qsm_sitk[lims[0]-1:lims[0]+lims[3]+1,lims[1]-1:lims[1]+lims[4]+1,lims[2]-1:lims[2]+lims[5]+1], 'qsm_crop_' + ids[j] +'_'+str(k)+ '.nrrd',True)

       




Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glszm': []}


In [6]:
# Import the maps from `.nrrd` files and visualize
feat_name = 'wavelet-HHL_glszm_SizeZoneNonUniformity'
r_num1 = 1
r_num2 = 4

In [7]:
X_r = (X_all_c[per_change>0.85,2:4,np.where(K_all_c == feat_name)[0][0]]).astype(float)
X_nr = (X_all_c[per_change<0.3,2:4,np.where(K_all_c == feat_name)[0][0]]).astype(float)
res = wilcoxon(X_r.ravel(),X_nr.ravel())
res.statistic, res.pvalue

(5.0, 0.3125)

In [8]:
np.median(X_r)

9.425675675675675

In [9]:
np.median(X_nr)

14.207122093023255

In [10]:
# ROI 1
pq = True
hr1,_ = nrrd.read('./'+feat_name+'_'+str(pr)+'_'+str(r_num1)+'.nrrd')
if pq == True:
    hrq1,_ = nrrd.read('./qsm_crop_'+str(pr)+'_'+str(r_num1)+'.nrrd')
    mr1,mrq1 = nrrd.read('./seg_crop_'+str(pr)+'_'+str(r_num1)+'.nrrd')
    # hr1 = hrq1*(abs(hr1)!=0)
    hr1 = hrq1

# ROI 2
hr2,_ = nrrd.read('./'+feat_name+'_'+str(pr)+'_'+str(r_num2)+'.nrrd')
if pq == True:
    hrq2,_ = nrrd.read('./qsm_crop_'+str(pr)+'_'+str(r_num2)+'.nrrd')
    mr2,mrq2 = nrrd.read('./seg_crop_'+str(pr)+'_'+str(r_num2)+'.nrrd')
    # hr2 = hrq2*(abs(hr2)!=0)
    hr2 = hrq2

# Pad the ROIs to the same shape
px,py,pz = np.asarray(hr1.shape)-np.asarray(hr2.shape)
px,py,pz
hr1p = np.pad(hr1,((0,abs(min(0,px))),(0,abs(min(0,py))),(0,abs(min(0,pz)))))
hr2p = np.pad(hr2,((0,abs(max(0,px))),(0,abs(max(0,py))),(0,abs(max(0,pz)))))
hr = np.flipud(np.concatenate((hr1p,hr2p),axis=2))
pyvis(hr,10,10,'gray',500,0,-250,250)

In [ ]:
# make_feature_map(res[2],feat_name,False,(r_num1,r_num2),8,0,0,3)
# make_feature_map(res[0],feat_name,False,(r_num1,r_num2),8,0,0,3)
# make_feature_map(nres[0],feat_name,False,(r_num1,r_num2),8,0,0,3)
# make_feature_map(nres[1],feat_name,False,(r_num1,r_num2),8,0,0,3)

In [ ]:
# make_feature_map(res[2],feat_name,True,(r_num1,r_num2),500,0,-250,250)
# make_feature_map(res[0],feat_name,True,(r_num1,r_num2),500,0,-250,250)
# make_feature_map(nres[0],feat_name,True,(r_num1,r_num2),500,0,-250,250)
# make_feature_map(nres[1],feat_name,True,(r_num1,r_num2),500,0,-250,250)